In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [2]:
# Load the MNIST dataset
digits = load_digits()
X = digits.images
y = digits.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data
X_train = X_train.reshape(-1, 8, 8, 1)
X_test = X_test.reshape(-1, 8, 8, 1)

# Normalize the input data
X_train = X_train / 16.0
X_test = X_test / 16.0

# Define the CNN architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(8, 8, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))



In [3]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
45/45 [==============================] - 1s 10ms/step - loss: 2.2353 - accuracy: 0.3152 - val_loss: 2.0962 - val_accuracy: 0.5306
Epoch 2/10
45/45 [==============================] - 0s 4ms/step - loss: 1.9133 - accuracy: 0.6903 - val_loss: 1.6882 - val_accuracy: 0.7750
Epoch 3/10
45/45 [==============================] - 0s 4ms/step - loss: 1.4370 - accuracy: 0.8205 - val_loss: 1.1523 - val_accuracy: 0.8528
Epoch 4/10
45/45 [==============================] - 0s 4ms/step - loss: 0.9614 - accuracy: 0.8803 - val_loss: 0.7784 - val_accuracy: 0.8806
Epoch 5/10
45/45 [==============================] - 0s 4ms/step - loss: 0.6618 - accuracy: 0.9012 - val_loss: 0.5521 - val_accuracy: 0.8917
Epoch 6/10
45/45 [==============================] - 0s 3ms/step - loss: 0.4989 - accuracy: 0.9186 - val_loss: 0.4482 - val_accuracy: 0.9111
Epoch 7/10
45/45 [==============================] - 0s 4ms/step - loss: 0.4054 - accuracy: 0.9235 - val_loss: 0.3645 - val_accuracy: 0.9083
Epoch 8/10
45/45 [=

In [4]:
# Evaluate the model
accuracy = model.evaluate(X_test, y_test)[1]
print("Accuracy:", accuracy)

12/12 [==============================] - 0s 2ms/step - loss: 0.2523 - accuracy: 0.9361
Accuracy: 0.9361110925674438


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 14 * 14, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        return x

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

# Create an instance of the CNN model
model = CNN().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} - Loss: {running_loss / len(trainloader)}")

# Test the model
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy: {accuracy}%")


100%|██████████| 9912422/9912422 [00:01<00:00, 7898652.62it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 28671170.14it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 5494318.82it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



Epoch 1 - Loss: 0.16281259355793395
Epoch 2 - Loss: 0.06886391291658704
Epoch 3 - Loss: 0.051461288820843526
Epoch 4 - Loss: 0.04099144617770022
Epoch 5 - Loss: 0.03415218327410985
